In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator()
cv_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

In [ ]:
import os

In [ ]:
training_datagen = train_datagen.flow_from_directory(directory="/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/Train",
                                                     target_size=(32,32),color_mode="grayscale",
                                                     classes=os.listdir("/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/Train"),
                                                     batch_size=100)

Found 62560 images belonging to 46 classes.


In [ ]:
validation_datagen = cv_datagen.flow_from_directory(directory="/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/cv",
                                                    target_size=(32,32),color_mode="grayscale",
                                                    classes=os.listdir("/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/cv"),
                                                    batch_size=15640)

Found 15640 images belonging to 46 classes.


In [ ]:
testing_datagen = test_datagen.flow_from_directory(directory="/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/Test",
                                                   target_size=(32,32),color_mode="grayscale",
                                                   classes=os.listdir("/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/Test"),
                                                   batch_size=13800)

Found 13800 images belonging to 46 classes.


In [ ]:
from tensorflow.keras.applications import vgg16

In [ ]:
conv_base = vgg16.VGG16(weights=None,include_top=False,input_shape=(32,32,1))

In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 1)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [ ]:
from tensorflow.keras.layers import Flatten,Dense,BatchNormalization
from tensorflow.keras.activations import relu,softmax
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.models import Sequential

In [ ]:
for l in conv_base.layers[::-1]:

    l.trainable = False

In [ ]:
def freeze_layers(layer):

  trainable = True

  for l in conv_base.layers[::-1]:

    if l.name == layer:
      l.trainable = False
      trainable = False

    l.trainable = trainable

    return conv_base

In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 1)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [ ]:
def build_network():

  network = Sequential()
  network.add(conv_base)
  network.add(Flatten())
  network.add(Dense(units=512,activation=relu))
  network.add(Dense(units=46,activation=softmax))

  return network

In [ ]:
network = build_network()

In [ ]:
network.compile(optimizer=Adam(),loss=categorical_crossentropy,metrics=["accuracy"])

In [ ]:
performance_history = network.fit_generator(generator=training_datagen,epochs=1,validation_data=validation_datagen,workers=2,use_multiprocessing=True)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


 43/626 [=>............................] - ETA: 1:58:32 - loss: 3.8205 - accuracy: 0.0319